In [1]:
!pip install flask

     |████████████████████████████████| 94 kB 2.1 MB/s eta 0:00:011
     |████████████████████████████████| 298 kB 9.0 MB/s eta 0:00:01


In [2]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [52]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="abc123")

[<Record n=<Node id=27 labels=frozenset({'ns0__ScholarlyArticle', 'Resource'}) properties={'ns1__doi': '10.1007/BF02165411', 'uri': 'file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/26209'}>>]

In [202]:
# query_string = '''
# MATCH (n:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'}) 
# OPTIONAL MATCH (n)-[r:ns1__references]->(a),
# OPTIONAL MATCH (n)-[r:ns0__citation]->(x)
# RETURN n,x,a'''
# query_string = query_string.replace('__PROPERTY__',search_property)
# query_string = query_string.replace('__TERM__',search_term)
# res = conn.query(query_string, db='neo4j')

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'OPTIONAL': expected "shortestPath", "allShortestPaths" or "(" (line 4, column 1 (offset: 113))
"OPTIONAL MATCH (n)-[r:ns0__citation]->(x)"
 ^}


In [234]:
# Get Paper Properties

def getPaper(search_property,search_term):
    dc = {}
    query_string = '''
    MATCH (n:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'}) RETURN n LIMIT 1'''
    
    query_string = query_string.replace('__PROPERTY__',search_property)
    query_string = query_string.replace('__TERM__',search_term)
    res = conn.query(query_string, db='neo4j')
    
    dc['info'] = res
    
    query_string = '''
    MATCH
    (:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns1__references]->(n)
    RETURN n.ns0__headline,n.ns1__doi,n.uri
    '''
    query_string = query_string.replace('__PROPERTY__',search_property)
    query_string = query_string.replace('__TERM__',search_term)
    dc['references'] = conn.query(query_string, db='neo4j')

    query_string = '''
    MATCH
    (:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns0__citation]->(n)
    RETURN n.ns0__headline,n.ns1__doi,n.uri
    '''
    query_string = query_string.replace('__PROPERTY__',search_property)
    query_string = query_string.replace('__TERM__',search_term)
    dc['citations'] = conn.query(query_string, db='neo4j')

    query_string = '''
    MATCH
    (:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns0__author]->(n)
    RETURN n
    '''
    query_string = query_string.replace('__PROPERTY__',search_property)
    query_string = query_string.replace('__TERM__',search_term)
    dc['authors'] = conn.query(query_string, db='neo4j')

    query_string = '''
    MATCH
    (:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns0__genre]->(n)
    RETURN n
    '''
    query_string = query_string.replace('__PROPERTY__',search_property)
    query_string = query_string.replace('__TERM__',search_term)
    dc['genres'] = conn.query(query_string, db='neo4j')

    query_string = '''
    MATCH
    (:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns0__publisher]->(n)
    RETURN n
    '''
    query_string = query_string.replace('__PROPERTY__',search_property)
    query_string = query_string.replace('__TERM__',search_term)
    dc['publisher'] = conn.query(query_string, db='neo4j')
    
    return dc

In [272]:
card_text_or = """						<div class="card" style="border-top:solid grey;">
								<div class="card-body">
								<br>
								<div class = "row">
									<div class = "col-2">
								<small class ="mr-4">Title</small>
							</div>

							<div class = "col-10"  >
								<h3>
__TITLE__</h3>
</div>
								<hr>
                                __PROPERTIES__
							</div>


									<div class = "d-flex">
									<p class="card-text mt-2 mr-auto"><small class="text-muted ">Results: __TOTAL__</small></p>
									<p class="card-text mt-2 ml-auto mr-auto"><small class="text-muted "> __NUMBER__</small></p>
											<p class="card-text mt-2 ml-auto"><small class="text-muted ">Executed at __TIME__</small></p></div>
											
							
								</div>
							
								</div><hr>"""

dict_keys(['ns1__doi', 'ns0__dateModified', 'ns0__headline', 'ns0__abstract', 'ns0__datePublished', 'ns0__url', 'uri', 'ns1__citationVelocity', 'ns1__b_influentialCitationCount'])

In [280]:
search_property = 'ns1__doi'
search_term = '10.1109/ICCV.2019.00500'
paper_dict = getPaper(search_property,search_term)
uri = paper_dict['info'][0]['n']['uri']

try:
    doi = paper_dict['info'][0]['n']['ns1__doi']
except:
    doi = None
try:
    title = paper_dict['info'][0]['n']['ns0__headline']
except:
    title = None

property_text = '<div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4">'+str(uri)+'</span></div>'

for i in paper_dict['info'][0]['n']:
    if i not in ["uri","ns0__abstract","ns0__headline"]:
        property_text+= '<div class = "col-2"><small >'+i+'</small></div><div class = "col-10"><span class="badge badge-info mr-4">'+str(paper_dict['info'][0]['n'][i])+'</span></div>'

if len(paper_dict['authors'])>0:
    property_text+= """<div class = "col-2"><small >Authors</small></div><div class = "col-10">"""
    for a in [i['n']['ns0__name'] for i in paper_dict['authors']]:
        property_text+='<span class="badge badge-info mr-4">'+a+'</span>'
    property_text+='</div>'
if len(paper_dict['genres'])>0:
    property_text+= """<div class = "col-2"><small >Genres</small></div><div class = "col-10">"""
    for a in [i['n']['ns0__name'] for i in paper_dict['genres']]:
        property_text+='<span class="badge badge-info mr-4">'+a+'</span>'
    property_text+='</div>'

if "ns0__abstract" in paper_dict['info'][0]['n']:
    property_text+= '<div class = "col-2 mt-4"><small >Abstract</small></div><div class = "col-10"><p">'+paper_dict['info'][0]['n']['ns0__abstract']+'</p></div>'

    
card_text = card_text_or[:]
card_text = card_text.replace('__TITLE__',title)
card_text = card_text.replace('__PROPERTIES__',property_text)

print(card_text)


						<div class="card" style="border-top:solid grey;">
								<div class="card-body">
								<br>
								<div class = "row">
									<div class = "col-2">
								<small class ="mr-4">Title</small>
							</div>

							<div class = "col-10"  >
								<h3>
Universal Perturbation Attack Against Image Retrieval</h3>
</div>
								<hr>
                                <div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4">file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/14598</span></div><div class = "col-2"><small >ns1__doi</small></div><div class = "col-10"><span class="badge badge-info mr-4">10.1109/ICCV.2019.00500</span></div><div class = "col-2"><small >ns0__dateModified</small></div><div class = "col-10"><span class="badge badge-info mr-4">2019</span></div><div class = "col-2"><small >ns0__headline</small></div><div class = "col-10"><span class="badge badge-info mr-4">Universal Perturbation Attack Against Imag

In [283]:
session = {'query_count':1}

In [295]:
cite_text = '<div class = "col-2"><small>Citations</small></div><div class = "col-10"><p><button class="btn btn-block btn-secondary" data-toggle="collapse" href="#collapseExampleC'+str(session["query_count"])+'" role="button" aria-expanded="false" aria-controls="collapseExample">'+str(len(citations))+'</button ></p><div class="collapse" id="collapseExampleC'+str(session["query_count"])+'"><div class="card card-body">__CITETEXT__</div></div></div>'

if len(paper_dict['citations'])>0:
    c_text = "<ul>"
    for i in paper_dict['citations']:
        c_text+= '<li><span class = "badge badge-primary mr-4">'+i['n.uri'].split("/")[-1]+'</span></li>'
    c_text+="</ul>"
    cite_text_2 = cite_text.replace('__CITETEXT__',c_text)
    property_text+=cite_text_2

cite_text = '<div class = "col-2"><small>Citations</small></div><div class = "col-10"><p><button class="btn btn-block btn-secondary" data-toggle="collapse" href="#collapseExampleR'+str(session["query_count"])+'" role="button" aria-expanded="false" aria-controls="collapseExample">'+str(len(references))+'</button ></p><div class="collapse" id="collapseExampleR'+str(session["query_count"])+'"><div class="card card-body">__CITETEXT__</div></div></div>'

if len(paper_dict['references'])>0:
    c_text = "<ul>"
    for i in paper_dict['references']:
        c_text+= '<li><span class = "badge badge-primary mr-4">'+i['n.uri'].split("/")[-1]+'</span></li>'
    c_text+="</ul>"
    cite_text_2 = cite_text.replace('__CITETEXT__',c_text)
    property_text+=cite_text_2

In [298]:
paper_dict['references'][0]['n.ns0__headline']

KeyError: 'n.s'

In [389]:
card_text_search = """						<div class="card" style="border-top:solid grey;">
								<div class="card-body">
								<br>
								<div class = "row">
									<div class = "col-2">
								<small class ="mr-4">Search __TERM__</small>
							</div>

							<div class = "col-10"  >
								<h3>
__TITLE__</h3>
</div>
								<hr>
                                __PROPERTIES__
							</div>


									<div class = "d-flex">
									<p class="card-text mt-2 mr-auto"><small class="text-muted ">Results: __TOTAL__</small></p>
									<p class="card-text mt-2 ml-auto mr-auto"><small class="text-muted "> __NUMBER__</small></p>
											<p class="card-text mt-2 ml-auto"><small class="text-muted ">Executed at __TIME__</small></p></div>
											
							
								</div>
							
								</div><hr>"""

In [390]:
search_term = 'NLP'

In [391]:
query_string = """MATCH (n:ns0__ScholarlyArticle {})
WHERE n.ns0__headline CONTAINS '__SEARCHTERM__'
RETURN n limit 10"""

query_string = query_string.replace('__SEARCHTERM__',search_term)

res = conn.query(query_string, db='neo4j')


In [392]:
card_text = card_text_search[:] 

In [393]:
property_text = ""
c_r = 0
for r in res:
    property_text+= '<div class = "col-12"><p><button class="btn btn-block btn-light text-left" data-toggle="collapse" href="#collapseExampleC'+str(session["query_count"])+'_'+str(c_r)+'" role="button" aria-expanded="false" aria-controls="collapseExample">‣'+r['n']['ns0__headline']+'</button ></p><div class="collapse" id="collapseExampleC'+str(session["query_count"])+'_'+str(c_r)+'"><div class="card card-body">'
    c_r+=1
    property_text+= '<div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4"><a style="color:white;" target="_blank" href="/page/2/'+str(r['n']['uri'].split("/")[-1])+'">'+str(r['n']['uri'].split("/")[-1])+'</a></span></div>'

    for i in r['n']:
        if i not in ["uri","ns0__abstract","ns0__headline"]:
            property_text+= '<div class = "col-2"><small >'+i+'</small></div><div class = "col-10"><span class="badge badge-info mr-4">'+str(r['n'][i])+'</span></div>'
    property_text+='</div></div></div>'
    
card_text = card_text.replace('__TERM__','TITLE')
card_text = card_text.replace('__TITLE__',search_term)
card_text = card_text.replace('__PROPERTIES__',property_text)
card_text = card_text.replace('__TOTAL__',len(res))



In [388]:
print(card_text)

						<div class="card" style="border-top:solid grey;">
								<div class="card-body">
								<br>
								<div class = "row">
									<div class = "col-2">
								<small class ="mr-4">Search __TERM__</small>
							</div>

							<div class = "col-10"  >
								<h3>
__TITLE__</h3>
</div>
								<hr>
                                <div class = "col-12"><p><button class="btn btn-block btn-light text-left" data-toggle="collapse" href="#collapseExampleC1_0" role="button" aria-expanded="false" aria-controls="collapseExample">‣Mixed-Precision Training for NLP and Speech Recognition with OpenSeq2Seq</button ></p><div class="collapse" id="collapseExampleC1_0"><div class="card card-body"><div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4"><a style="color:white;" href="/page/2/17892">17892</a></span></div><div class = "col-2"><small >ns0__dateModified</small></div><div class = "col-10"><span class="badge badge-info mr-4">2018</span></di

In [373]:
property_text

'<div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4">file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/17892</span></div><div class = "col-2"><small >ns0__dateModified</small></div><div class = "col-10"><span class="badge badge-info mr-4">2018</span></div><div class = "col-2"><small >ns0__datePublished</small></div><div class = "col-10"><span class="badge badge-info mr-4">2018</span></div><div class = "col-2"><small >ns0__url</small></div><div class = "col-10"><span class="badge badge-info mr-4">http://arxiv.org/abs/1805.10387v2</span></div><div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4">file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/20015</span></div><div class = "col-2"><small >ns0__dateModified</small></div><div class = "col-10"><span class="badge badge-info mr-4">2017</span></div><div class = "col-2"><small >ns0__datePublished</small></d

In [343]:
for i in res[0]['n']:
    print(res[0]['n'][i])

2018
Mixed-Precision Training for NLP and Speech Recognition with OpenSeq2Seq
We present OpenSeq2Seq - a TensorFlow-based toolkit for trainingsequence-to-sequence models that features distributed and mixed-precisiontraining. Benchmarks on machine translation and speech recognition tasks showthat models built using OpenSeq2Seq give state-of-the-art performance at 1.5-3xless training time. OpenSeq2Seq currently provides building blocks for modelsthat solve a wide range of tasks including neural machine translation,automatic speech recognition, and speech synthesis.
2018
http://arxiv.org/abs/1805.10387v2
file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/17892


In [306]:
if len(res)>0:
    c_text = "<ul>"
    for i in res:
        c_text+= '<li><span class = "badge badge-primary mr-4">'+i['n']['uri'].split("/")[-1]+'</span>'
        if i['n']['ns0__headline']:
            c_text+=i['n']['ns0__headline']+'</li>'
        else:
            c_text+='</li>'

    c_text+="</ul>"


[<Record n=<Node id=7548 labels=frozenset({'ns0__ScholarlyArticle', 'Resource'}) properties={'ns0__dateModified': '2018', 'ns0__headline': 'Mixed-Precision Training for NLP and Speech Recognition with OpenSeq2Seq', 'ns0__abstract': 'We present OpenSeq2Seq - a TensorFlow-based toolkit for trainingsequence-to-sequence models that features distributed and mixed-precisiontraining. Benchmarks on machine translation and speech recognition tasks showthat models built using OpenSeq2Seq give state-of-the-art performance at 1.5-3xless training time. OpenSeq2Seq currently provides building blocks for modelsthat solve a wide range of tasks including neural machine translation,automatic speech recognition, and speech synthesis.', 'ns0__datePublished': '2018', 'ns0__url': 'http://arxiv.org/abs/1805.10387v2', 'uri': 'file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/17892'}>>,
 <Record n=<Node id=15284 labels=frozenset({'ns0__ScholarlyArticle', 'Resource'}) properties={'ns0__dateMod

In [287]:
citations[0].keys()

['n.ns0__headline', 'n.ns1__doi', 'n.uri']

'<div class = "col-2"><small>Citations</small></div><div class = "col-10"><p><button class="btn btn-block btn-secondary" data-toggle="collapse" href="#collapseExample1" role="button" aria-expanded="false" aria-controls="collapseExample">5</button ></p><div class="collapse" id="collapseExample1"><div class="card card-body">__CITETEXT__</div></div></div>'

In [278]:
paper_dict['info'][0]['n'][i]

7

In [259]:
# property_text = '<div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4">'+str(uri)+'</span></div>'
# property_text+= '<div class = "col-2"><small >DOI</small></div><div class = "col-10"><span class="badge badge-info mr-4">'+str(doi)+'</span></div>'

# property_text+= """<div class = "col-2"><small >Authors</small></div><div class = "col-10">"""
# for a in [i['n']['ns0__name'] for i in paper_dict['authors']]:
#     property_text+='<span class="badge badge-info mr-4">'+a+'</span>'
# property_text+='</div>'

# property_text+= """<div class = "col-2"><small >Genres</small></div><div class = "col-10">"""
# for a in [i['n']['ns0__name'] for i in paper_dict['authors']]:
#     property_text+='<span class="badge badge-info mr-4">'+a+'</span>'
# property_text+='</div>'



In [262]:
print(card_text)

						<div class="card" style="border-top:solid grey;">
								<div class="card-body">
								<br>
								<div class = "row">
									<div class = "col-2">
								<small class ="mr-4">Title</small>
							</div>

							<div class = "col-10"  >
								<h3>
Reconstructing Capsule Networks for Zero-shot Intent Classification</h3>
</div>
								<hr>
                                <div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4">file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/3595</span></div><div class = "col-2"><small >DOI</small></div><div class = "col-10"><span class="badge badge-info mr-4">10.18653/v1/D19-1486</span></div><div class = "col-2"><small >Authors</small></div><div class = "col-10"><span class="badge badge-info mr-4">刘晗</span><span class="badge badge-info mr-4">Qimai Li</span><span class="badge badge-info mr-4">Xuandi Fu</span><span class="badge badge-info mr-4">Lu Fan</span><span class="badg

In [ ]:
for i in 

In [ ]:
"""<div class = "col-2">

								<small >Authors</small>

							</div>
							<div class = "col-10">"""

In [239]:
paper_dict['references'][0]

<Record n.ns0__headline=None n.ns1__doi='10.1109/ICASSP.2012.6289054' n.uri='file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/21185'>

In [240]:
paper_dict['info']

[<Record n=<Node id=463572 labels=frozenset({'ns0__ScholarlyArticle', 'Resource'}) properties={'ns1__doi': '10.18653/v1/D19-1486', 'ns0__headline': 'Reconstructing Capsule Networks for Zero-shot Intent Classification', 'ns0__abstract': 'Intent classification is an important building block of dialogue systems. With the burgeoning of conversational AI, existing systems are not capable of handling numerous fast-emerging intents, which motivates zero-shot intent classification. Nevertheless, research on this problem is still in the incipient stage and few methods are available. A recently proposed zero-shot intent classification method, IntentCapsNet, has been shown to achieve state-of-the-art performance. However, it has two unaddressed limitations: (1) it cannot deal with polysemy when extracting semantic capsules; (2) it hardly recognizes the utterances of unseen intents in the generalized zero-shot intent classification setting. To overcome these limitations, we propose to reconstruct 

In [92]:
query_string = '''
MATCH (n:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'}) RETURN n LIMIT 1'''
query_string = query_string.replace('__PROPERTY__',search_property)
query_string = query_string.replace('__TERM__',search_term)

In [145]:
genre

[]

In [ ]:
query_string = '''
MATCH
(:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns]->(n)
RETURN n.ns0__headline,n.ns1__doi,n.uri
'''
query_string = query_string.replace('__PROPERTY__',search_property)
query_string = query_string.replace('__TERM__',search_term)
ls = conn.query(query_string, db='neo4j')

In [126]:
citations

[<Record n.ns0__headline='Text Classification Using Label Names Only: A Language Model Self-Training Approach' n.ns1__doi='10.18653/v1/2020.emnlp-main.724' n.uri='file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/11001'>,
 <Record n.ns0__headline='Unknown Intent Detection Using Gaussian Mixture Model with an Application to Zero-shot Intent Classification' n.ns1__doi='10.18653/v1/2020.acl-main.99' n.uri='file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/10997'>,
 <Record n.ns0__headline='Topic-Enhanced Capsule Network for Multi-Label Emotion Classification' n.ns1__doi='10.1109/TASLP.2020.3001390' n.uri='file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/1863'>,
 <Record n.ns0__headline='Dynamic Semantic Matching and Aggregation Network for Few-shot Intent Detection' n.ns1__doi='10.18653/v1/2020.findings-emnlp.108' n.uri='file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/10995'>,
 <Record n.ns0__headline='BDARS_CapsNe

In [94]:
paper

[<Record n=<Node id=463572 labels=frozenset({'ns0__ScholarlyArticle', 'Resource'}) properties={'ns1__doi': '10.18653/v1/D19-1486', 'ns0__headline': 'Reconstructing Capsule Networks for Zero-shot Intent Classification', 'ns0__abstract': 'Intent classification is an important building block of dialogue systems. With the burgeoning of conversational AI, existing systems are not capable of handling numerous fast-emerging intents, which motivates zero-shot intent classification. Nevertheless, research on this problem is still in the incipient stage and few methods are available. A recently proposed zero-shot intent classification method, IntentCapsNet, has been shown to achieve state-of-the-art performance. However, it has two unaddressed limitations: (1) it cannot deal with polysemy when extracting semantic capsules; (2) it hardly recognizes the utterances of unseen intents in the generalized zero-shot intent classification setting. To overcome these limitations, we propose to reconstruct 

In [99]:
references[0]

<Record n.ns0__headline=None n.ns1__doi='10.1109/ICASSP.2012.6289054' n.uri='file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/21185'>

In [65]:
query_string = '''
MATCH
(:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns0__citations]->(n)
RETURN n.ns0__headline,n.ns1__doi LIMIT 25
'''
query_string = query_string.replace('__PROPERTY__',search_property)
query_string = query_string.replace('__TERM__',search_term)

citations = conn.query(query_string, db='neo4j')

<Record n=<Node id=463572 labels=frozenset({'ns0__ScholarlyArticle', 'Resource'}) properties={'ns1__doi': '10.18653/v1/D19-1486', 'ns0__headline': 'Reconstructing Capsule Networks for Zero-shot Intent Classification', 'ns0__abstract': 'Intent classification is an important building block of dialogue systems. With the burgeoning of conversational AI, existing systems are not capable of handling numerous fast-emerging intents, which motivates zero-shot intent classification. Nevertheless, research on this problem is still in the incipient stage and few methods are available. A recently proposed zero-shot intent classification method, IntentCapsNet, has been shown to achieve state-of-the-art performance. However, it has two unaddressed limitations: (1) it cannot deal with polysemy when extracting semantic capsules; (2) it hardly recognizes the utterances of unseen intents in the generalized zero-shot intent classification setting. To overcome these limitations, we propose to reconstruct c

In [71]:
i['n'].keys()

dict_keys(['ns1__doi', 'ns0__headline', 'ns0__abstract', 'ns0__datePublished', 'ns0__url', 'uri', 'ns1__citationVelocity', 'ns1__b_influentialCitationCount'])

In [ ]:
query_string = '''
MATCH
(:ns0__ScholarlyArticle {__PROPERTY__:'__TERM__'})-[r:ns1__references]->(n)
RETURN n.ns0__headline,n.ns1__doi LIMIT 25
'''
query_string = query_string.replace('__PROPERTY__',search_property)
query_string = query_string.replace('__TERM__',search_term)

In [336]:
query_string = '''MATCH (n:ns0__ScholarlyArticle {uri:'14598'}) RETURN n LIMIT 1'''

res = conn.query(query_string, db='neo4j')

In [337]:
res

[<Record n=<Node id=102296 labels=frozenset({'ns0__ScholarlyArticle', 'Resource'}) properties={'ns1__doi': '10.1109/ICCV.2019.00500', 'ns0__dateModified': '2019', 'ns0__headline': 'Universal Perturbation Attack Against Image Retrieval', 'ns0__abstract': 'Universal adversarial perturbations (UAPs), a.k.a. input-agnostic perturbations, has been proved to exist and be able to fool cutting-edge deep learning models on most of the data samples. Existing UAP methods mainly focus on attacking image classification models. Nevertheless, little attention has been paid to attacking image retrieval systems. In this paper, we make the first attempt in attacking image retrieval systems. Concretely, image retrieval attack is to make the retrieval system return irrelevant images to the query at the top ranking list. It plays an important role to corrupt the neighbourhood relationships among features in image retrieval attack. To this end, we propose a novel method to generate retrieval-against UAP to 

# Author Extraction

In [403]:
!pip install sparql-client

     |████████████████████████████████| 224 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 188 kB 17.1 MB/s eta 0:00:01
  Created wheel for sparql-client: filename=sparql_client-3.8-py3-none-any.whl size=13664 sha256=1aaea1c925c84f891de481917ea454048dba8daf99586d29a606f0d4d314b28b
  Stored in directory: /Users/rehanahmed/Library/Caches/pip/wheels/69/5a/f6/a73593e360435b07635d40cc3abe01d4bf5fc0c27daa4b2820
Successfully built sparql-client


In [398]:
query_string = '''
MATCH (n:ns0__Person) 
RETURN n'''
res = conn.query(query_string, db='neo4j')

In [399]:
len(res)

50677

In [491]:
author_names = [i['n']['ns0__name']+","+i['n']['uri'] for i in res]

In [494]:
with open('authors.txt','w+') as f:
    f.write('\n'.join(author_names))
    f.close()

In [ ]:
with open('author_names')

In [407]:
import sparql

In [473]:
sparql_query_or = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbo:<http://dbpedia.org/ontology/>

SELECT ?lbl ?sub ?a ?b ?c ?d WHERE {
  ?sub a foaf:Person .
  ?sub rdfs:label ?lbl .
  ?lbl bif:contains "__TERM__" .
  filter(langMatches(lang(?lbl), "en"))
OPTIONAL {?sub dbo:birthName ?a}
OPTIONAL {?sub dbo:birthPlace ?b}
OPTIONAL {?sub dbo:birthDate ?c}
?sub dbo:academicDiscipline ?d

} 
limit 100
"""

In [474]:
ans_dict = {}

In [481]:
from tqdm import tqdm

In [482]:
c=0
for r in tqdm(res):
    c+=1

    name_res = r['n']['ns0__name'].split(" ")
    # name_res = "G. Hinton".split(" ")
    search_name = ""
    search_bool = True

    if len(name_res)>1:
        for i in name_res:
            search_name = search_name +i+' AND '
            if len(i)<2 or i[1]==".":
                search_bool = False

        if search_bool:
            sparql_query = sparql_query_or[:]
            sparql_query = sparql_query.replace('__TERM__',search_name[:-5])
            result = sparql.query('http://dbpedia.org/sparql', sparql_query)

            ans_list = []
            for row in result:
                values = sparql.unpack_row(row)
                ans_list.append(values)
            if len(ans_list)>0:
                ans_dict[r['n']['uri']] = ans_list


  0%|          | 101/50677 [00:33<4:42:40,  2.98it/s]


KeyboardInterrupt: 

In [485]:
print(ans_dict['file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/1048565'][0])

['Chen Hao-wei', 'http://dbpedia.org/resource/Chen_Hao-wei', None, 'http://dbpedia.org/resource/Hualien_County', datetime.date(1992, 4, 30)]


In [446]:
search_bool

False

In [496]:
import pickle

In [499]:
import os

In [500]:
os.listdir()

['.DS_Store',
 'Neo4J Queries.txt',
 'Website Testing.ipynb',
 'pagerank',
 'ans_dict.pickle',
 'synthethic_abstracts.csv',
 'setup.py',
 'ans_dict (1).pickle',
 'static',
 'templates',
 '.ipynb_checkpoints',
 'neo4j_queries.txt',
 'authors.txt']

In [534]:
authors = pickle.load(open('ans_dict (1).pickle','rb'))

In [535]:
for i in authors:
    print(authors[i])
    break

[['Choh Hao Li', 'http://dbpedia.org/resource/Choh_Hao_Li', None, 'http://dbpedia.org/resource/Guangzhou', datetime.date(1913, 4, 21), 'http://dbpedia.org/resource/Biochemist'], ['Choh Hao Li', 'http://dbpedia.org/resource/Choh_Hao_Li', None, 'http://dbpedia.org/resource/China', datetime.date(1913, 4, 21), 'http://dbpedia.org/resource/Biochemist'], ['Hao Li', 'http://dbpedia.org/resource/Hao_Li', None, 'http://dbpedia.org/resource/Saarbrücken', datetime.date(1981, 1, 17), 'http://dbpedia.org/resource/Computer_Graphics'], ['Hao Li', 'http://dbpedia.org/resource/Hao_Li', None, 'http://dbpedia.org/resource/West_Germany', datetime.date(1981, 1, 17), 'http://dbpedia.org/resource/Computer_Graphics'], ['Hao Li', 'http://dbpedia.org/resource/Hao_Li', None, 'http://dbpedia.org/resource/Saarbrücken', datetime.date(1981, 1, 17), 'http://dbpedia.org/resource/Computer_Vision'], ['Hao Li', 'http://dbpedia.org/resource/Hao_Li', None, 'http://dbpedia.org/resource/West_Germany', datetime.date(1981, 1, 

In [542]:
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD

MYNS = Namespace('http://inf558.org/myfakenamespace#')
SCHEMA = Namespace("https://schema.org/")

# Initliaze the graph
g = rdflib.Graph()

# Bind namespace and prefixes
g.bind('my_ns', MYNS)
g.bind('schema', SCHEMA)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('xsd', XSD)

for d in tqdm(authors):
    node_uri = URIRef(d.split("/")[-1])
    g.add((node_uri, RDF.type, SCHEMA.Person))
    g.add((node_uri, SCHEMA.name, Literal(authors[d][0][0], datatype=SCHEMA.Text)))
    if authors[d][0][4]:
        g.add((node_uri, SCHEMA.birthDate, Literal(authors[d][0][4], datatype=SCHEMA.Date)))
    g.add((node_uri, SCHEMA.url, Literal(authors[d][0][1], datatype=SCHEMA.URL)))

    

100%|██████████| 298/298 [00:00<00:00, 8598.31it/s]


In [543]:
print(g.serialize( format="turtle").decode())

@prefix schema: <https://schema.org/> .

<1000058> a schema:Person ;
    schema:name "Ding Yu Peng"^^schema:Text ;
    schema:url "http://dbpedia.org/resource/Ding_Yu_Peng"^^schema:URL .

<1000423> a schema:Person ;
    schema:name "Peter Wonka"^^schema:Text ;
    schema:url "http://dbpedia.org/resource/Peter_Wonka"^^schema:URL .

<1000438> a schema:Person ;
    schema:name "John Milton Miller"^^schema:Text ;
    schema:url "http://dbpedia.org/resource/John_Milton_Miller"^^schema:URL .

<1000465> a schema:Person ;
    schema:name "Gang Hua"^^schema:Text ;
    schema:url "http://dbpedia.org/resource/Gang_Hua"^^schema:URL .

<1000554> a schema:Person ;
    schema:birthDate "1923-01-17"^^schema:Date ;
    schema:name "David Shepherd Nivison"^^schema:Text ;
    schema:url "http://dbpedia.org/resource/David_Shepherd_Nivison"^^schema:URL .

<1000727> a schema:Person ;
    schema:birthDate "1921-05-20"^^schema:Date ;
    schema:name "Hao Wang (academic)"^^schema:Text ;
    schema:url "http://

In [538]:
authors

{'file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/1047868': [['Choh Hao Li',
   'http://dbpedia.org/resource/Choh_Hao_Li',
   None,
   'http://dbpedia.org/resource/Guangzhou',
   datetime.date(1913, 4, 21),
   'http://dbpedia.org/resource/Biochemist'],
  ['Choh Hao Li',
   'http://dbpedia.org/resource/Choh_Hao_Li',
   None,
   'http://dbpedia.org/resource/China',
   datetime.date(1913, 4, 21),
   'http://dbpedia.org/resource/Biochemist'],
  ['Hao Li',
   'http://dbpedia.org/resource/Hao_Li',
   None,
   'http://dbpedia.org/resource/Saarbrücken',
   datetime.date(1981, 1, 17),
   'http://dbpedia.org/resource/Computer_Graphics'],
  ['Hao Li',
   'http://dbpedia.org/resource/Hao_Li',
   None,
   'http://dbpedia.org/resource/West_Germany',
   datetime.date(1981, 1, 17),
   'http://dbpedia.org/resource/Computer_Graphics'],
  ['Hao Li',
   'http://dbpedia.org/resource/Hao_Li',
   None,
   'http://dbpedia.org/resource/Saarbrücken',
   datetime.date(1981, 1, 17),
   'http:/

In [546]:
search_term = 'Geoffrey'

In [547]:
query_string = """MATCH (n:ns0__Person {})
WHERE toLower(n.ns0__name) CONTAINS toLower('__SEARCHTERM__')
RETURN n limit 10"""

query_string = query_string.replace('__SEARCHTERM__',search_term)

res = conn.query(query_string, db='neo4j')
card_text = card_text_or[:] 

property_text = '<hr class="mt-4">'
c_r = 0
for r in res:
    property_text+= '<div class = "col-12"><p><button class="btn btn-block btn-light text-left" data-toggle="collapse" href="#collapseExampleC'+str(session["query_count"])+'_'+str(c_r)+'" role="button" aria-expanded="false" aria-controls="collapseExample">‣ '+r['n']['ns0__name']+'</button ></p><div class="collapse" id="collapseExampleC'+str(session["query_count"])+'_'+str(c_r)+'"><div class="card card-body">'
    c_r+=1
    property_text+= '<div class = "row"><div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4"><a style="color:white;" target="_blank" href="/page/2/'+str(r['n']['uri'].split("/")[-1])+'">'+str(r['n']['uri'].split("/")[-1])+'</a></span></div></div>'

    for i in r['n']:
        if i not in ["uri","ns0__name"]:
            property_text+= '<div class = "row"><div class = "col-2"><small >'+i+'</small></div><div class = "col-10"><span class="badge badge-info mr-4">'+str(r['n'][i])+'</span></div></div>'
    property_text+='</div></div></div>'

card_text = card_text.replace('Title','Search Title')
card_text = card_text.replace('__TITLE__',search_term)
card_text = card_text.replace('__PROPERTIES__',property_text)
card_text = card_text.replace('__TOTAL__',str(len(res)))

In [551]:
search_term = "file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/"+str(1050403)

In [555]:
query_string = '''
MATCH
(x:ns0__Person {uri:'__TERM__'})<-[r:ns0__author]-(n)
RETURN n
'''
query_string = query_string.replace('__TERM__',search_term)

res = conn.query(query_string, db='neo4j')
card_text = card_text_or[:] 

property_text = '<hr class="mt-4">'
c_r = 0
for r in res:
    property_text+= '<div class = "col-12"><p><button class="btn btn-block btn-light text-left" data-toggle="collapse" href="#collapseExampleC'+str(session["query_count"])+'_'+str(c_r)+'" role="button" aria-expanded="false" aria-controls="collapseExample">‣ '+r['n']['ns0__headline']+'</button ></p><div class="collapse" id="collapseExampleC'+str(session["query_count"])+'_'+str(c_r)+'"><div class="card card-body">'
    c_r+=1
    property_text+= '<div class = "row"><div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4"><a style="color:white;" target="_blank" href="/page/2/'+str(r['n']['uri'].split("/")[-1])+'">'+str(r['n']['uri'].split("/")[-1])+'</a></span></div></div>'

    for i in r['n']:
        if i not in ["uri","ns0__abstract","ns0__headline"]:
            property_text+= '<div class = "row"><div class = "col-2"><small >'+i+'</small></div><div class = "col-10"><span class="badge badge-info mr-4">'+str(r['n'][i])+'</span></div></div>'
    property_text+='</div></div></div>'

card_text = card_text.replace('Title','Search Title')
card_text = card_text.replace('__TITLE__',search_term)
card_text = card_text.replace('__PROPERTIES__',property_text)
card_text = card_text.replace('__TOTAL__',str(len(res)))

'\t\t\t\t\t\t<div class="card" style="border-top:solid grey;">\n\t\t\t\t\t\t\t\t<div class="card-body">\n\t\t\t\t\t\t\t\t<br>\n\t\t\t\t\t\t\t\t<div class = "row">\n\t\t\t\t\t\t\t\t\t<div class = "col-2">\n\t\t\t\t\t\t\t\t<small class ="mr-4">Search Title</small>\n\t\t\t\t\t\t\t</div>\n\n\t\t\t\t\t\t\t<div class = "col-10"  >\n\t\t\t\t\t\t\t\t<h3>\nfile:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/1050403</h3>\n</div>\n\t\t\t\t\t\t\t\t<hr>\n                                <hr class="mt-4"><div class = "col-12"><p><button class="btn btn-block btn-light text-left" data-toggle="collapse" href="#collapseExampleC1_0" role="button" aria-expanded="false" aria-controls="collapseExample">‣ Visual attribute transfer through deep image analogy</button ></p><div class="collapse" id="collapseExampleC1_0"><div class="card card-body"><div class = "row"><div class = "col-2"><small >URI</small></div><div class = "col-10"><span class="badge badge-info mr-4"><a style="color:white;" targe

In [573]:
query_string = search_term

res = conn.query(query_string, db='neo4j')
card_text = card_text_or[:] 

property_text = '<hr class="mt-4">'
property_text+= '<div class="card card-body">'

if res:
    for r in res:
        property_text+='<div class = "row">'+str(r)+'</div>'
    property_text+='</div>'

    card_text = card_text.replace('Title','Query')
    card_text = card_text.replace('__PROPERTIES__',property_text)
    card_text = card_text.replace('__TOTAL__',str(len(res)))

else:
    card_text = card_text.replace('__PROPERTIES__','Error: Incorrect Query')
    card_text = card_text.replace('__TOTAL__',str(0))

card_text = card_text.replace('__TITLE__',search_term)


Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'i': expected 'r/R' or 'o/O' (line 1, column 2 (offset: 1))
"file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/1050403"
  ^}


In [574]:
card_text

'\t\t\t\t\t\t<div class="card" style="border-top:solid grey;">\n\t\t\t\t\t\t\t\t<div class="card-body">\n\t\t\t\t\t\t\t\t<br>\n\t\t\t\t\t\t\t\t<div class = "row">\n\t\t\t\t\t\t\t\t\t<div class = "col-2">\n\t\t\t\t\t\t\t\t<small class ="mr-4">Title</small>\n\t\t\t\t\t\t\t</div>\n\n\t\t\t\t\t\t\t<div class = "col-10"  >\n\t\t\t\t\t\t\t\t<h3>\nfile:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/1050403</h3>\n</div>\n\t\t\t\t\t\t\t\t<hr>\n                                Error: Incorrect Query\n\t\t\t\t\t\t\t</div>\n\n\n\t\t\t\t\t\t\t\t\t<div class = "d-flex">\n\t\t\t\t\t\t\t\t\t<p class="card-text mt-2 mr-auto"><small class="text-muted ">Results: 0</small></p>\n\t\t\t\t\t\t\t\t\t<p class="card-text mt-2 ml-auto mr-auto"><small class="text-muted "> __NUMBER__</small></p>\n\t\t\t\t\t\t\t\t\t\t\t<p class="card-text mt-2 ml-auto"><small class="text-muted ">Executed at __TIME__</small></p></div>\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t</div>\n\t\t\t\t\t\t\t\n\t\

# Synthetic Abstracts

In [576]:
import pandas as pd

In [578]:
os.listdir('../data')

['semantic_joined_dblp.json.zip',
 'Matches_test.csv',
 'complete_merged.pkl',
 '.DS_Store',
 'Merged_graphics.csv',
 'papers.ttl',
 'semantic_joined_dblp.json',
 'papers_4.ttl',
 'Matches.csv',
 'Merged_ai.csv',
 'papers_5.ttl',
 'final_dataset.csv',
 'output.json',
 'semantic_scholar_articles_20000.csv',
 'papers_2.ttl',
 'papers_5.ttl.zip',
 'papers_3.ttl',
 'Merged_vision.csv',
 'final_dataset.pkl',
 'synthethic_abstracts.csv',
 'Merged_nlp.csv',
 'ans_dict (1).pickle',
 'output_compiled.json',
 'authors.ttl',
 'authors.txt',
 'complete_merged.csv',
 'semantic_short.csv',
 'output.txt']

In [586]:
import csv

In [627]:
with open('../data/Neo4J Queries.txt') as f:
    t=f.read()
    f.close()

In [631]:
ls_t = t.split('_____')

In [642]:
ls_t[1]

'\n// Paper with Most Out Links \nMATCH (b:ns0__ScholarlyArticle)\nWITH b, SIZE((b)-[:ns1__references]->()) as referenceCount\nORDER BY referenceCount DESC LIMIT 1\nRETURN b\n'

In [645]:

q_num = 1
ans_global =""
for i in ls_t:
    i = i.replace('\n',' ')
    query_name = i.split('MATCH')[0][3:].strip()
    query = i

    ans='<div class = "row"><br><button class = "btn btn-light btn-block" id ="query__NUMBER__">__NAME__<p id = "query__NUMBER___hidden" style = "display:none">__QUERY__</p></button></div>'
    ans = ans.replace('__NUMBER__',str(q_num))
    ans = ans.replace('__NAME__',query_name)
    ans = ans.replace('__QUERY__',query)

    ans_global+=ans

print( ans_global)

<div class = "row"><br><button class = "btn btn-light btn-block" id ="query1">Paper with Most In Links<p id = "query1_hidden" style = "display:none">// Paper with Most In Links MATCH (b:ns0__ScholarlyArticle) WITH b, SIZE(()-[:ns1__references]->(b)) as referenceCount ORDER BY referenceCount DESC LIMIT 1 RETURN b </p></button></div><div class = "row"><br><button class = "btn btn-light btn-block" id ="query1">Paper with Most Out Links<p id = "query1_hidden" style = "display:none"> // Paper with Most Out Links  MATCH (b:ns0__ScholarlyArticle) WITH b, SIZE((b)-[:ns1__references]->()) as referenceCount ORDER BY referenceCount DESC LIMIT 1 RETURN b </p></button></div><div class = "row"><br><button class = "btn btn-light btn-block" id ="query1">Most Connected Node with Connections<p id = "query1_hidden" style = "display:none"> // Most Connected Node with Connections MATCH (b:ns0__ScholarlyArticle) WITH b, SIZE(()-[:ns1__references]->(b)) as referenceCount ORDER BY referenceCount DESC LIMIT 1 

In [644]:
query_name

' Longest Abstract '

In [598]:
ls=[]
with open('../data/synthethic_abstracts.csv', newline='',encoding='utf-8' ) as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t', quotechar='"')
    for row in spamreader:
        try:
            ls.append(row)
        except:
            pass
        

In [602]:
query_string = '''
MATCH (n:ns0__ScholarlyArticle) 
RETURN n.uri'''
res = conn.query(query_string, db='neo4j')

In [604]:
get

In [606]:
for d in ls[1:]:
    print(d)
    break

['18177', '16659', '18871', '8701', '4125', 'Dialog state tracking', 'a machine reading approach using Memory Network', 'Exploring Models and Data for Image Question Answering', 'Deep Multi-scale Convolutional Neural Network for Dynamic Scene  Deblurring', 'Two-Stream Multi-Channel Convolutional Neural Network for Multi-Lane Traffic Speed Prediction Considering Traffic Volume Impact', 'A sparse deep belief network with efficient fuzzy learning frameworkThe algorithm is based on a deep neural network that is able to learn to distinguish between different types of blurred scenes . The algorithm can be applied to any scene, including static and dynamic ones, and it can also be used to improve the performance of existing algorithms . The authors demonstrate the use of deep learning in traffic prediction by using a network trained on real traffic data to train a model that could predict how fast a particular vehicle would be traveling at any given time of day, based on its speed in relation

'A sparse deep belief network with efficient fuzzy learning frameworkThe algorithm is based on a deep neural network that is able to learn to distinguish between different types of blurred scenes . The algorithm can be applied to any scene, including static and dynamic ones, and it can also be used to improve the performance of existing algorithms . The authors demonstrate the use of deep learning in traffic prediction by using a network trained on real traffic data to train a model that could predict how fast a particular vehicle would be traveling at any given time of day, based on its speed in relation to other vehicles, such as cars, trucks, buses, motorcycles, bicycles, etc. In this way, the researchers demonstrate how deep networks can be used in a wide range of problems, including traffic management, vehicle safety, traffic congestion, urban planning, transportation planning and transportation planning . They hope that this work will inspire other researchers to explore the pote

In [611]:
c_uri = 453463

In [618]:
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD

MYNS = Namespace('http://inf558.org/myfakenamespace#')
SCHEMA = Namespace("https://schema.org/")

# Initliaze the graph
g = rdflib.Graph()

# Bind namespace and prefixes
g.bind('my_ns', MYNS)
g.bind('schema', SCHEMA)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('xsd', XSD)

for d in ls[1:]:
    node_uri = URIRef(str(c_uri))
    g.add((node_uri, RDF.type, SCHEMA.ScholarlyArticle))
    g.add((node_uri, SCHEMA.abstract, Literal(d[-1], datatype=SCHEMA.Text)))
    
    for i in d[0:5]:
        g.add((node_uri, SCHEMA.creditText, URIRef(str(i))))
    
    c_uri += 1

In [620]:
g.serialize('../data/synthetic_papers.ttl', format="turtle")

In [617]:
ls[5]

['14179',
 '19307',
 '19945',
 '4035',
 '13084',
 'A review on video-based active and assisted living technologies for automated lifelogging',
 'Densely Connected Convolutional Networks for Speech Recognition',
 'A robust self-learning method for fully unsupervised cross-lingual  mappings of word embeddings',
 'Unsupervised Segmentation of Images using CNN',
 'Interactive Deep Colorization Using Simultaneous Global and Local Inputs',
 'Recent work has managed to learn cross-lingual word embeddings withoutparallel data by mapping monolingual . words to a shared space throughadversarial training . It is propagated back to parent track to adjust the network parameters such that the loss will decrease with each iteration. Our results show that our approach is able to achieve a significant improvement in the accuracy of our network over the previous state-of-the-art. However, their evaluation has focused on favorable conditions, using comparable corpora or closely-related languages, and we 

In [660]:
query_string = "MATCH (n)-[r:ns0__creditText]->() RETURN DISTINCT n.uri LIMIT 5"

In [661]:
res = conn.query(query_string, db='neo4j')

In [662]:
res[0]['n.uri']

'file:///Users/rehanahmed/Documents/USC/DSCI-558%20Project/notebooks/453517'

In [663]:
query_string_2 = '''
MATCH
(:ns0__ScholarlyArticle {uri:'__TERM__'})-[r:ns0__creditText]->(n)
RETURN n
'''
query_string_2=query_string_2.replace('__TERM__',res[0]['n.uri'])
res_2 = conn.query(query_string_2, db='neo4j')

card_text = card_text_or[:]

if len(paper_dict['citations'])>0:
    c_text = "<ul>"
    for i in paper_dict['citations']:
        c_text+= '<li><span class = "badge badge-primary mr-4" ><a style="color:white;" target = "_blank" href="/page/2/'+i['n.uri'].split("/")[-1]+'">'+i['n.uri'].split("/")[-1]+'</a></span>'
        if i['n.ns0__headline']:
            c_text+=i['n.ns0__headline']+'</li>'
        else:
            c_text+='</li>'

    c_text+="</ul>"
    cite_text_2 = cite_text.replace('__CITETEXT__',c_text)
    property_text+=cite_text_2

5